In [1]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path().resolve().parent.parent))

In [3]:
import time
import triton
import triton.language as tl
import torch
import torch.nn.functional as F
import pandas as pd
from conv_gemm.triton_kernels.fp16.img2col_kernel import img2col_kernel

In [4]:
def img2col_fp16(
    x,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    BLOCK_M,
    BLOCK_K,
    num_warps,
    num_stages,
):
    assert x.is_cuda
    assert x.dtype in (torch.float16, torch.float32)

    N, Cin, H, W = x.shape

    Ho = (H + 2*Ph - Dh*(Kh - 1) - 1)//Sh + 1
    Wo = (W + 2*Pw - Dw*(Kw - 1) - 1)//Sw + 1

    M = N * Ho * Wo
    K = Cin * Kh * Kw

    cols = torch.empty((M, K), dtype=x.dtype, device=x.device)
    sN, sC, sH, sW = x.stride()

    grid = (triton.cdiv(M, BLOCK_M), triton.cdiv(K, BLOCK_K))

    img2col_kernel[grid](
        x, cols,
        N, Cin, H, W,
        Kh, Kw, Sh, Sw, Ph, Pw, Dh, Dw,
        Ho, Wo,
        sN, sC, sH, sW,
        K,
        BLOCK_M=BLOCK_M,
        BLOCK_K=BLOCK_K,
        CAST_FP16=(x.dtype == torch.float16),
        num_warps=num_warps,
        num_stages=num_stages,
    )

    return cols, (Ho, Wo)


In [5]:
@torch.no_grad()
def bench_once_img2col_fp16_vs_unfold(
    N, Cin, H, W,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    BLOCK_M,
    BLOCK_K,
    num_warps,
    num_stages,
    iters=100,
    device="cuda",
):

    x = torch.randn((N, Cin, H, W), device=device, dtype=torch.float16)
    x_f32 = x.float()

    Ho = (H + 2 * Ph - Dh * (Kh - 1) - 1) // Sh + 1
    Wo = (W + 2 * Pw - Dw * (Kw - 1) - 1) // Sw + 1

    K = Cin * Kh * Kw
    M = N * Ho * Wo

    # ----- TORCH UNFOLD -----
    def _call_torch():
        return F.unfold(
            x_f32,
            kernel_size=(Kh, Kw),
            dilation=(Dh, Dw),
            padding=(Ph, Pw),
            stride=(Sh, Sw),
        )

    # warmup
    for _ in range(5):
        _call_torch()
    torch.cuda.synchronize()

    # timing
    t0 = time.perf_counter()
    for _ in range(iters):
        unf = _call_torch()
    torch.cuda.synchronize()
    t_torch = (time.perf_counter() - t0) / iters

    # torch output → [M, K]
    unf_M_K = unf.permute(0, 2, 1).contiguous().view(M, K)

    # ----- TRITON FP16 IMG2COL -----
    def _call_triton():
        cols, (Ho2, Wo2) = img2col_fp16(
            x,
            Kh, Kw,
            Sh, Sw,
            Ph, Pw,
            Dh, Dw,
            BLOCK_M=BLOCK_M,
            BLOCK_K=BLOCK_K,
            num_warps=num_warps,
            num_stages=num_stages,
        )
        assert Ho2 == Ho and Wo2 == Wo
        return cols

    for _ in range(5):
        _call_triton()
    torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        cols = _call_triton()
    torch.cuda.synchronize()
    t_triton = (time.perf_counter() - t0) / iters

    # bandwidth (FP16 → 2 bytes)
    bytes_moved = 2.0 * M * K * 2   # load + store * fp16(2 bytes)
    bw_triton = bytes_moved / t_triton / 1e9
    bw_torch  = bytes_moved / t_torch / 1e9

    return {
        "M": M, "K": K, "N": N,
        "BLOCK_M": BLOCK_M,
        "BLOCK_N": 0,
        "BLOCK_K": BLOCK_K,
        "num_warps": num_warps,
        "num_stages": num_stages,
        "t_triton_ms": t_triton * 1e3,
        "t_torch_ms": t_torch * 1e3,
        "speed_vs_torch": t_torch / t_triton,
        "bw_triton_GBs": bw_triton,
        "bw_torch_GBs": bw_torch,
    }

In [6]:
@torch.no_grad()
def tune_img2col_fp16_tiles_for_shape(
    N, Cin, H, W,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    blocks_M=(32, 64, 128),
    blocks_K=(32, 64, 128),
    warps=(1, 2, 4, 8),
    stages=(2, 3, 4),
    iters=200,
    device="cuda",
):
    records = []
    for BM in blocks_M:
        for BK in blocks_K:
            for W in warps:
                for S in stages:
                    try:
                        rec = bench_once_img2col_fp16_vs_unfold(
                            N, Cin, H, W,
                            Kh, Kw,
                            Sh, Sw,
                            Ph, Pw,
                            Dh, Dw,
                            BLOCK_M=BM,
                            BLOCK_K=BK,
                            num_warps=W,
                            num_stages=S,
                            iters=iters,
                            device=device,
                        )
                    except RuntimeError as e:
                        print(f"[SKIP] BM={BM}, BK={BK}, W={W}, S={S}: {e}")
                        continue

                    print(
                        f"BM={BM}, BK={BK}, W={W}, S={S}: "
                        f"t_triton={rec['t_triton_ms']:.3f} ms, "
                        f"speed_vs_torch={rec['speed_vs_torch']:.3f}x, "
                    )
                    records.append(rec)

    if not records:
        raise RuntimeError("No valid tile configs found for this IMG2COL shape")

    df = pd.DataFrame(records)
    return df


In [7]:
df_fp16_i2c = tune_img2col_fp16_tiles_for_shape(
    N=16, Cin=1, H=256, W=256,
    Kh=11, Kw=11,
    Sh=1, Sw=1,
    Ph=5, Pw=5,
    Dh=1, Dw=1,
    blocks_M=(32, 64, 128),
    blocks_K=(32, 64, 128),
    warps=(2, 4, 8, 16),
    stages=(2, 3),
    iters=200,
    device="cuda",
)


BM=32, BK=32, W=2, S=2: t_triton=0.037 ms, speed_vs_torch=5.570x, 
BM=32, BK=32, W=2, S=3: t_triton=0.035 ms, speed_vs_torch=5.903x, 
BM=32, BK=32, W=4, S=2: t_triton=0.038 ms, speed_vs_torch=9.928x, 
BM=32, BK=32, W=4, S=3: t_triton=0.036 ms, speed_vs_torch=10.858x, 
BM=32, BK=32, W=8, S=2: t_triton=0.049 ms, speed_vs_torch=7.429x, 
BM=32, BK=32, W=8, S=3: t_triton=0.049 ms, speed_vs_torch=7.542x, 
BM=32, BK=32, W=16, S=2: t_triton=0.118 ms, speed_vs_torch=2.974x, 
BM=32, BK=32, W=16, S=3: t_triton=0.119 ms, speed_vs_torch=3.118x, 
BM=32, BK=64, W=2, S=2: t_triton=0.036 ms, speed_vs_torch=5.828x, 
BM=32, BK=64, W=2, S=3: t_triton=0.037 ms, speed_vs_torch=5.294x, 
BM=32, BK=64, W=4, S=2: t_triton=0.037 ms, speed_vs_torch=10.107x, 
BM=32, BK=64, W=4, S=3: t_triton=0.035 ms, speed_vs_torch=11.028x, 
BM=32, BK=64, W=8, S=2: t_triton=0.047 ms, speed_vs_torch=7.611x, 
BM=32, BK=64, W=8, S=3: t_triton=0.044 ms, speed_vs_torch=8.621x, 
BM=32, BK=64, W=16, S=2: t_triton=0.093 ms, speed_vs_torc

In [8]:
df_fp16_i2c.sort_values("t_triton_ms").head(10)

,M,K,N,BLOCK_M,BLOCK_N,BLOCK_K,num_warps,num_stages,t_triton_ms,t_torch_ms,speed_vs_torch,bw_triton_GBs,bw_torch_GBs
16,8192,121,16,32,0,128,2,2,0.033389,0.206752,6.192199,118.749422,19.177261
51,16384,121,16,128,0,32,4,3,0.034255,0.359010,10.480519,231.495287,22.088150
59,16384,121,16,128,0,64,4,3,0.034264,0.369291,10.777742,231.432319,21.473173
42,16384,121,16,64,0,128,4,2,0.034521,0.372997,10.804972,229.711740,21.259819
57,8192,121,16,128,0,64,2,3,0.034532,0.193900,5.615034,114.818053,20.448328
11,16384,121,16,32,0,64,4,3,0.034575,0.381285,11.027776,229.352307,20.797693
58,16384,121,16,128,0,64,4,2,0.034892,0.370574,10.620714,227.271141,21.398857
1,8192,121,16,32,0,32,2,3,0.034926,0.206163,5.902894,113.524264,19.231968
48,8192,121,16,128,0,32,2,2,0.035008,0.197147,5.631522,113.258481,20.111522
17,8192,121,16,32,0,128,2,3,0.035668,0.201533,5.650253,111.161910,19.673793


In [9]:
FP16_IMG2COL_BEST_BLOCK_M = 32
FP16_IMG2COL_BEST_BLOCK_N = 0
FP16_IMG2COL_BEST_BLOCK_K = 128
FP16_IMG2COL_BEST_WARPS   = 2
FP16_IMG2COL_BEST_STAGES  = 2